<a href="https://colab.research.google.com/github/Palaeoprot/IPA/blob/main/Phenylalanine_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Example of reaction pathways
Data taken from

Changi, S., Zhu, M. and Savage, P.E. (2012) ‘Hydrothermal reaction kinetics and pathways of phenylalanine alone and in binary mixtures’, ChemSusChem, 5(9), pp. 1743–1757.

<img src='https://drive.google.com/uc?export=view&id=1ph38QmGoK5k6Uaiu9DFcwh1qJadJAVtl' width=800px align=centre>



In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import curve_fit
import seaborn as sns

# Constants
R = 8.314  # Gas constant in J/mol·K

#('Qualitative',
                    #  ['Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2',
                    #   'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b',
                    #   'tab20c'])

sns.set_palette("Dark2")


# Arrhenius parameters
arrhenius_params = [
    {"A": 10**12.5, "Ea": 146000},  # Reaction 1
    {"A": 10**7.7, "Ea": 113000},   # Reaction 2
#    {"A": 10**10.6, "Ea": 155000},  # Reaction 3
    {"A": 10**13.9, "Ea": 153000},  # Reaction 4
#    {"A": 10**7.3, "Ea": 132000},   # Reaction 5
#    {"A": 10**8.5, "Ea": 140000},   # Reaction 6
]

# Temperature range from 0°C to 250°C
temperatures_C = np.linspace(0, 140, 10)
temperatures_K = temperatures_C + 273.15

# Calculate rate constants for each reaction at each temperature
rate_constants = np.zeros((len(arrhenius_params), len(temperatures_K)))

for i, params in enumerate(arrhenius_params):
    A = params["A"]
    Ea = params["Ea"]
    rate_constants[i] = A * np.exp(-Ea / (R * temperatures_K))

# Determine the observed rate (fastest rate at each temperature)
observed_rate_constants = np.max(rate_constants, axis=0)

# Fit the apparent rate to a pseudo-first-order irreversible reaction
def arrhenius_eq(T, A, Ea):
    return A * np.exp(-Ea / (R * T))

# Provide a reasonable initial guess for A and Ea
# The values below are just examples and should be adjusted based on the data
p0 = [1e10, 1e5]
popt, _ = curve_fit(arrhenius_eq, temperatures_K, observed_rate_constants, p0=p0, maxfev=10000)  # Increase maxfev
A_fit, Ea_fit = popt

# Plotting
plt.figure(figsize=(10, 6))

# Plot the data
for i, params in enumerate(arrhenius_params):
    plt.plot(1/temperatures_K, np.log(rate_constants[i]), label=f'Reaction {i+1}')

# Plot the observed rate
plt.plot(1/temperatures_K, np.log(observed_rate_constants), label='Dominant Rate', linewidth=2, linestyle='--', color='black')

# Plot fitted apparent rate
fitted_rate_constants = arrhenius_eq(temperatures_K, A_fit, Ea_fit)
plt.plot(1/temperatures_K, np.log(fitted_rate_constants), label='Fitted Apparent Rate', linewidth=2, linestyle=':', color='red')


# Set the title, labels, and legend
plt.xlabel('1/T (1/K)')
plt.ylabel('ln(k)')
plt.suptitle('Arrhenius Plot for Phenylalanine Reaction Network', fontsize=14)
plt.title('Data from Changi et al. 2007, Table 4. Only the three fastest reactions are shown for simplicity', fontsize=10)
plt.legend()
plt.grid(True)
plt.show()

